Code for  model training was the result of the following paper:

CANet: Cross-Disease Attention Network for Joint Diabetic Retinopathy and Diabetic Macular Edema Grading


In [ ]:
import gc
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Importing Libraries

In [ ]:
import torch
from PIL import Image
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.models as models
import tensorflow
import tensorboard
import random
import numpy as np
import argparse
import os
import shutil
import time
import sys
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import label_binarize
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import confusion_matrix
import scipy.misc
from skimage.transform import resize
#imports from drive
# sys.path.append('/content/drive/MyDrive/lr_scheduler.py')  
%cd '/content/drive/MyDrive'
import lr_scheduler as l_rate
import missidor
# import average_result
import resnet50
# !nvidia-smi
# print(torch.__version__)

/content/drive/MyDrive


File write 

writes the dataset image names to a file so run only once

In [ ]:
# !unzip -uq '/content/drive/My Drive/Base.zip' -d '/content/drive/My Drive/Base'
# fileName=os.listdir('/content/drive/My Drive/Base/Base')
# total=len(fileName)
# f = open('file_list.txt','a')
# for i in range(0,total):
#   f.write(fileName[i])
#   f.write(" ")
# f.close()

In [ ]:

my_whole_seed = 111
torch.manual_seed(my_whole_seed)
torch.cuda.manual_seed_all(my_whole_seed)
torch.cuda.manual_seed(my_whole_seed)
np.random.seed(my_whole_seed)
random.seed(my_whole_seed)
cudnn.deterministic = True
cudnn.benchmark = False
model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))

Class AverageMeter

In [ ]:
class AverageMeter(object):

  """Computes and stores the average and current value"""
  def __init__(self):
    self.reset()

  def reset(self):
    self.val = 0
    self.avg = 0
    self.sum = 0
    self.count = 0

  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count += n
    self.avg = self.sum / self.count

In [ ]:
class identifyDataset(data.Dataset):
  def __init__(self,imageName,transform):

    self.imageName = imageName
    self.transform = transform

  def __len__(self):
    return len(self.imageName)

  def __getitem__(self, idx):
    img = Image.open(self.imageName)
    image = img.convert('RGB')
    image = self.transform(image)
    return image

        

Class CANet


In [ ]:
class CANet:
  def __init__(self):

    self.data = '/content/drive/My Drive/Base/Base/' #path to dataset
    self.dataset = "missidor"
    self.model_dir = '/content/drive/My Drive/Trained/'
    self.a = 'resnet50'
    self.arch = 'resnet50' #default 'alexnet'
    self.j = 4
    self.workers = 4
    self.epochs = 1000 #default 300
    self.num_class = 2
    self.start_epoch = 0
    self.batch_size = 40 #default 20
    self.lr = 0.1
    self.learning_rate = 0.1
    self.momentum = 0.9
    self.wd = 1e-4
    self.liu = False
    self.weight_decay = 1e-4
    self.p = 10
    self.print_freq = 10
    #e , evaluate boolean evaluates model on validation set
    self.resume=''
    self.e = False
    self.evaluate = False
    self.pretrained = True #boolean uses pretrained model
    self.decay_epoch = 500 #default 15
    self.seed = 111 #for initializing training
    self.gpu = 0 #GPU id to use
    #multitask , liu , chen , crossCBAM , adam , choice stores boolean
    self.adam = True
    self.crossCBAM = True
    self.multitask = True
    self.chen = False
    self.crosspatialCBAM = False
    self.CAN_TS = False
    self.choice = "both"
    self.net_type = "regular"
    self.channels = 109
    self.nodes = 32
    self.graph_model = "WS"
    self.K = 4
    self.P = 0.75
    self.fold_name = "fold0"
    self.fold_name2 = "fold2"
    self.fold_name3 = "fold3"
    #lr.......
    self.lr_mode = "cosine"
    self.base_lr = 3e-4 #default 0.03
    self.warmup_epochs = 0
    self.warmup_lr = 0.0
    self.targetlr = 0.0
    self.lambda_value = 0.25
    self.best_acc1 = 0
    self.best_auc = 0
    self.best_accdr = 0
    self.minimum_loss = 1
    self.count = 0
    pass

  def count_parameters(self,model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    pass

  def worker_init_fn(self,worker_id):
    random.seed(1 + worker_id)
    pass  
    
  def softmax(self,x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

  # def save_checkpoint(self,state, is_best, filename='checkpoint.pth', save_dir= 'file'):
    
  #   root = save_dir + "/"
  #   if not os.path.exists(root):
  #       os.makedirs(root)
  #   torch.save(state, root+filename)
  #   pass

  # def save_result2txt(self,savedir, all_output_dme, all_output,all_target_dme,all_target):

  #   np.savetxt(savedir+"/output_dme.txt", all_output_dme, fmt='%.4f')
  #   np.savetxt(savedir+"/output_dr.txt", all_output, fmt='%.4f')
  #   np.savetxt(savedir+"/target_dme.txt", all_target_dme)
  #   np.savetxt(savedir+"/target_dr.txt", all_target)
  #   pass
  
  def multi_class_auc(self,all_target, all_output, num_c = None):

    # multi-class accuraccy
    all_output = np.stack(all_output)
    all_target = label_binarize(all_target, classes=list(range(0, num_c)))
    auc_sum = []
    for num_class in range(0, num_c):
      try:
        auc = roc_auc_score(all_target[:, num_class], all_output[:, num_class])
        auc_sum.append(auc)
      except ValueError:
        pass

    auc = sum(auc_sum) / float(len(auc_sum))
    return auc

  def save_result_txt(self,savedir, result):

    with open(savedir + '/result1(1).txt', 'w') as f:
      for item in result:
        f.write("%.8f\n" % item)
      f.close()

  def train(self,train_loader, model, criterion, lr_scheduler, writer, epoch, optimizer):
    
    print("Training...")

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()

    # switch to train mode
    model.train()
    end = time.time()

    for i, (input, target, name) in enumerate(train_loader):
      # measure data loading time
      data_time.update(time.time() - end)
      lr = lr_scheduler.update(i, epoch)
      writer.add_scalar("lr", lr, epoch)
      if   self.gpu is not None:
        input = input.cuda(  self.gpu, non_blocking = True)
      if   self.multitask:
        target = [item.cuda(  self.gpu, non_blocking = True) for item in target]
      else:
        target = target.cuda(  self.gpu, non_blocking = True)

      # compute output
      output = model(input)
      if   self.multitask:
        loss1 = criterion(output[0], target[0])
        loss2 = criterion(output[1], target[1])
        if   self.crossCBAM:
          loss3 = criterion(output[2], target[0])
          loss4 = criterion(output[3], target[1])
          loss = (loss1 + loss2 +   self.lambda_value *loss3 +   self.lambda_value * loss4)
        else:
          loss = (loss1 + loss2)
      else:
        loss = criterion(output, target)
      
      losses.update(loss.item(), input.size(0))

      # compute gradient and do SGD step
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # measure elapsed time
      batch_time.update(time.time() - end)
      end = time.time()

    return losses.avg

  def validate(self,val_loader, model):

    # switch to evaluate mode
    print("Validating...")
    model.eval()
    all_target = []
    all_target_dme = []
    all_output = []
    all_name = []
    all_output_dme = []
    # writer = SummaryWriter(self.model_dir)
    # writer.add_text('Text', "Comparision of labels")
    # writer.add_text("Text", "Predicted on axis-y")
    # writer.add_text("Text", "Truth on axis-x")
    with torch.no_grad():
      for i, (input, target, name) in enumerate(val_loader):
        if   self.gpu is not None:
          input = input.cuda(  self.gpu, non_blocking=True)
        if   self.multitask:
          target = [item.cuda(  self.gpu, non_blocking=True) for item in target]
        else:
          target = target.cuda(  self.gpu, non_blocking=True)
        print("Image Shape:  ",input.shape)
        output = model(input)
        torch.cuda.synchronize()
        if   self.multitask:
          output0 = output[0] #dr
          output1 = output[1] #dme
          output0 = torch.softmax(output0, dim=1)
          output1 = torch.softmax(output1, dim=1)
          all_target.append(target[0].cpu().data.numpy())
          all_output.append(output0.cpu().data.numpy())
          all_target_dme.append(target[1].cpu().data.numpy())
          all_output_dme.append(output1.cpu().data.numpy())
        else:
          output = torch.softmax(output, dim=1)
          all_target.append(target.cpu().data.numpy())
          all_output.append(output.cpu().data.numpy())
        all_name.append(name)
    
    if   self.dataset == "kaggle":
      all_output = [item for sublist in all_output for item in sublist]
      all_target = [item for sublist in all_target for item in sublist]
      acc_dr = accuracy_score(all_target, np.argmax(all_output, axis=1))
      auc_dr = multi_class_auc(all_target, all_output, num_c=5)
      return acc_dr, auc_dr
    elif not   self.multitask:
      all_target = [item for sublist in all_target for item in sublist]
      all_output = [item for sublist in all_output for item in sublist]
      if   self.num_class == 2:
        acc = accuracy_score(all_target, np.argmax(all_output,axis=1))
        auc = roc_auc_score(all_target, [item[1] for item in all_output])
        precision_dr = precision_score(all_target, np.argmax(all_output, axis=1))
        recall_dr = recall_score(all_target, np.argmax(all_output, axis=1))
        f1score_dr = f1_score(all_target, np.argmax(all_output, axis=1))
      else:
        acc = accuracy_score(all_target, np.argmax(all_output, axis=1))
        auc = self.multi_class_auc(all_target, all_output, num_c=3)
        precision_dr = precision_score(all_target, np.argmax(all_output, axis=1), average="macro")
        recall_dr = recall_score(all_target, np.argmax(all_output, axis=1), average="macro")
        f1score_dr = f1_score(all_target, np.argmax(all_output, axis=1), average="macro")
      return acc, auc, precision_dr, recall_dr, f1score_dr
    else:
      all_target = [item for sublist in all_target for item in sublist]
      all_output = [item for sublist in all_output for item in sublist]
      all_target_dme = [item for sublist in all_target_dme for item in sublist]
      all_output_dme = [item for sublist in all_output_dme for item in sublist]
      # acc
      acc_dr = accuracy_score(all_target, np.argmax(all_output,axis=1)) #1st arg predicted label
      acc_dme = accuracy_score(all_target_dme, np.argmax(all_output_dme, axis=1))
      # pr_dme = np.argmax(all_output_dme,axis=1)
      # tr_dme = np.argmax(all_target_dme)
      # writer.add_scalar('Label DME', pr_dme[0], tr_dme)
    
      # joint acc
      joint_result = np.vstack((np.argmax(all_output, axis=1), np.argmax(all_output_dme, axis=1)))
      joint_target = np.vstack((all_target, all_target_dme))
      joint_acc = ((np.equal(joint_result, joint_target) == True).sum(axis=0) == 2).sum() / joint_result.shape[1]
      # auc
      if   self.dataset == "missidor":
        auc_dr  = roc_auc_score(all_target, [item[1] for item in all_output])
      else:
        auc_dr = self.multi_class_auc(all_target, all_output, num_c = 5)
      auc_dme = self.multi_class_auc(all_target_dme, all_output_dme, num_c = 3)
      # precision
      if   self.dataset == "missidor":
        precision_dr = precision_score(all_target, np.argmax(all_output, axis=1))
      else:
        precision_dr = precision_score(all_target, np.argmax(all_output, axis=1), average="macro")
      precision_dme = precision_score(all_target_dme, np.argmax(all_output_dme, axis=1), average="macro")
      # recall
      if   self.dataset == "missidor":
        recall_dr = recall_score(all_target, np.argmax(all_output, axis=1))
      else:
        recall_dr  = recall_score(all_target, np.argmax(all_output, axis=1), average="macro")
      recall_dme = recall_score(all_target_dme, np.argmax(all_output_dme, axis=1), average="macro")
      # f1_score
      if   self.dataset == "missidor":
        f1score_dr = f1_score(all_target, np.argmax(all_output, axis=1))
      else:
        f1score_dr = f1_score(all_target, np.argmax(all_output, axis=1), average="macro")
      f1score_dme = f1_score(all_target_dme, np.argmax(all_output_dme, axis=1), average="macro")

      cm1 = confusion_matrix(all_target, np.argmax(all_output, axis=1))
      sensitivity1 = cm1[0, 0] / (cm1[0, 0] + cm1[0, 1])
      specificity1 = cm1[1, 1] / (cm1[1, 0] + cm1[1, 1])
      
      # writer.close()
      return acc_dr, acc_dme, joint_acc, \
            [auc_dr, auc_dme, precision_dr, precision_dme, recall_dr, recall_dme, f1score_dr, f1score_dme],\
            sensitivity1, specificity1

  def main_worker(self):

    if not os.path.exists(  self.model_dir):
      os.makedirs(  self.model_dir)

    if   self.gpu is not None:
      print("Use GPU: {} for training".format(  self.gpu))

    if   self.arch == "resnet50":
      #from models.resnet50 import resnet50
      model = resnet50.resnet50(num_classes=  self.num_class, multitask=  self.multitask, liu=  self.liu,
                chen=  self.chen, CAN_TS=  self.CAN_TS, crossCBAM=  self.crossCBAM,
                crosspatialCBAM =   self.crosspatialCBAM,  choice=  self.choice)

    if   self.pretrained:
      model_dict = model.state_dict()
      pretrain_path = {"resnet50": "/content/drive/My Drive/resnet50-19c8e357.pth",}[  self.arch]
      pretrained_dict = torch.load(pretrain_path)
      pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
      pretrained_dict.pop('classifier.weight', None)
      pretrained_dict.pop('classifier.bias', None)
      model_dict.update(pretrained_dict)
      model.load_state_dict(model_dict)

    torch.cuda.set_device(  self.gpu)
    model = model.cuda(  self.gpu)

    print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters()) / 1000000.0))

    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda(  self.gpu)
    if   self.adam:
      optimizer = torch.optim.Adam(model.parameters(),   self.base_lr, weight_decay=  self.weight_decay)
    else:
      optimizer = torch.optim.SGD(model.parameters(),   self.base_lr,
                              momentum=  self.momentum,
                              weight_decay=  self.weight_decay)

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    size  = 224
    tra = transforms.Compose([
                transforms.Resize(256),
                transforms.RandomResizedCrop(size),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                # transforms.RandomRotation(90),
                # transforms.ColorJitter(0.05, 0.05, 0.05, 0.05),
                transforms.ToTensor(),
                normalize,
            ])
    
    train_dataset = missidor.traindataset(root=  self.data, mode='train', transform=tra, num_class=  self.num_class,
                                multitask=  self.multitask,   args =  self)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=  self.batch_size, shuffle=True,
        num_workers=  self.workers, pin_memory=True,worker_init_fn=self.worker_init_fn)

    writer = SummaryWriter(self.model_dir + "runs")
    writer.add_text('Text', "CANet")
    #
    #from lr_scheduler import LRScheduler
    lr_scheduler = l_rate.LRScheduler(optimizer, len(train_loader),self)

    for epoch in range(  self.start_epoch,   self.epochs):
      is_best = False
      is_best_auc = False
      is_best_acc = False
      # train for one epoch
      loss_train = self.train(train_loader, model, criterion, lr_scheduler, writer, epoch, optimizer)
      template = 'Epoch {}, Loss: {}'
      print (template.format(epoch+1,
              loss_train))
      writer.add_scalar('Train loss', loss_train, epoch)
    writer.close()

    torch.save({'epoch': self.epochs,
                              'state_dict': model.state_dict(),
                              'optim_dict' : optimizer.state_dict()}, self.model_dir + "CANet(3).pth",_use_new_zipfile_serialization=False)
    
  def EvaluateModel(self):
    self.resume = '/content/drive/MyDrive/Trained/CANet(2).pth'
    self.evaluate = True
    self.e = True

    if   self.arch == "resnet50":
      #from models.resnet50 import resnet50
      model = resnet50.resnet50(num_classes=  self.num_class, multitask=  self.multitask, liu=  self.liu,
                chen=  self.chen, CAN_TS=  self.CAN_TS, crossCBAM=  self.crossCBAM,
                crosspatialCBAM =   self.crosspatialCBAM,  choice=  self.choice)

    if   self.pretrained:
      
      model_dict = model.state_dict()
      pretrain_path = {"resnet50": "/content/drive/My Drive/resnet50-19c8e357.pth",}[  self.arch]
      pretrained_dict = torch.load(pretrain_path)
      pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
      pretrained_dict.pop('classifier.weight', None)
      pretrained_dict.pop('classifier.bias', None)
      model_dict.update(pretrained_dict)
      model.load_state_dict(model_dict)

    torch.cuda.set_device(  self.gpu)
    model = model.cuda(  self.gpu)

    if   self.resume:
      if os.path.isfile(  self.resume):
        print("=> loading checkpoint '{}'".format(  self.resume))
        checkpoint = torch.load(  self.resume)
        #  load partial weights
        if not   self.evaluate:
          print ("load partial weights")
          model_dict = model.state_dict()
          pretrained_dict = {k: v for k, v in checkpoint['state_dict'].items() if k in model_dict}
          model_dict.update(pretrained_dict)
          model.load_state_dict(model_dict)
        else:
          print("loading weights")
          model.load_state_dict(checkpoint['state_dict'])
          # optimizer.load_state_dict(checkpoint['optim_dict'])
      else:
        print("=> no checkpoint found at '{}'".format(  self.resume))
        exit(0)
    
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
    
    tra_test = transforms.Compose([
          transforms.Resize(256),
          transforms.CenterCrop(224),
          transforms.ToTensor(),
          normalize])
    val_dataset = missidor.traindataset(root=  self.data, mode = 'val',
                            transform=tra_test, num_class=  self.num_class,
                            multitask=  self.multitask,   args =  self)
    val_loader = torch.utils.data.DataLoader(
      val_dataset,
      batch_size=  self.batch_size, shuffle=False,
      num_workers=  self.workers, pin_memory=True)
    
    if   self.evaluate:
      a = time.time()
      savedir = self.model_dir
      if not   self.multitask:
        acc, auc, precision_dr, recall_dr, f1score_dr  = self.validate(val_loader, model)
        result_list = [acc, auc, precision_dr, recall_dr, f1score_dr]
        print ("acc, auc, precision, recall, f1", acc, auc, precision_dr, recall_dr, f1score_dr)
        self.save_result_txt(savedir, result_list)
        print("time", time.time() - a)
        return
      else:
        acc_dr, acc_dme, acc_joint, other_results, se, sp = self.validate(val_loader, model)
        print ("acc_dr, acc_dme, acc_joint", acc_dr, acc_dme, acc_joint)
        print ("auc_dr, auc_dme, precision_dr, precision_dme, recall_dr, recall_dme, f1score_dr, f1score_dme",
                other_results)
        print ("se, sp", se, sp)
        result_list = [acc_dr, acc_dme, acc_joint]
        result_list += other_results
        result_list += [se, sp]
        self.save_result_txt(savedir, result_list)

        print ("time", time.time()-a)
        return
    
  def identify(self,name):
    all_output = []
    all_output_dme = []
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    tra_test = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize])
    
    path = '/content/drive/My Drive/' + name
    imageTra = identifyDataset(imageName=path,transform=tra_test)
    img_loader = torch.utils.data.DataLoader(
        imageTra,
        batch_size=  1, shuffle=False,
        num_workers=  0, pin_memory=True)
    
    model = resnet50.resnet50(num_classes=  2, multitask=  False, liu=  False,
                chen=  False, CAN_TS=  False, crossCBAM=  True,
                crosspatialCBAM =   False,  choice=  False)
    
    model_dict = model.state_dict()
    pretrain_path = {"resnet50": "/content/drive/My Drive/resnet50-19c8e357.pth",}[  self.arch]
    pretrained_dict = torch.load(pretrain_path)
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    pretrained_dict.pop('classifier.weight', None)
    pretrained_dict.pop('classifier.bias', None)
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)
    CUDA_VISIBLE_DEVICES=""
    # torch.cuda.set_device(  self.gpu)
    # model = model.cuda(  self.gpu)
    # optimizer = torch.optim.Adam(model.parameters(),   self.base_lr, weight_decay=  self.weight_decay)
    checkpoint = torch.load('/content/drive/MyDrive/Trained/CANet(2).pth',map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['state_dict'])
    # optimizer.load_state_dict(checkpoint['optim_dict'])
    print("model loaded")
    model.eval()
    model.to('cpu')
    
    with torch.no_grad():

      # for i, (input) in enumerate(img_loader):
      #   print(i)

        # if  self.gpu is not None:
        #   input = input.cuda(  self.gpu, non_blocking=True)

      data_iter=iter(img_loader)
      input = data_iter.next()
      output = model(input)
      # torch.cuda.synchronize()
      output0 = output[0] #dr
      output1 = output[1] #dme
      output0 = torch.softmax(output0, dim=1)
      output1 = torch.softmax(output1, dim=1)
      all_output.append(output0.cpu().data.numpy())
      all_output_dme.append(output1.cpu().data.numpy())
      all_output = [item for sublist in all_output for item in sublist]
      all_output_dme = [item for sublist in all_output_dme for item in sublist]
      pr_dr = np.argmax(all_output,axis=1)
      pr_dme = np.argmax(all_output_dme,axis=1)

      print("Name: ", name ,"DR_stage : ", pr_dr[0])
      print("Name: ", name ,"DME_stage : ", pr_dme[0])



Main

In [ ]:
# 70 30 split
#for dataset of 1200 ==> 360 test & 840 training
#for dataset of 800 ==> 240 test & 560 training
test = CANet()
#for training------
#test.main_worker() 
#for model validation------
test.EvaluateModel()  
#for subject image stage identification------
# test.identify('IDRiD_006.jpg') 


=> loading checkpoint '/content/drive/MyDrive/Trained/CANet(2).pth'
loading weights
=> Total Test:  240  Multi-Task images 
Validating...
Image Shape:   torch.Size([40, 3, 224, 224])
Image Shape:   torch.Size([40, 3, 224, 224])
Image Shape:   torch.Size([40, 3, 224, 224])
Image Shape:   torch.Size([40, 3, 224, 224])
Image Shape:   torch.Size([40, 3, 224, 224])
Image Shape:   torch.Size([40, 3, 224, 224])
acc_dr, acc_dme, acc_joint 0.8458333333333333 0.9 0.7833333333333333
auc_dr, auc_dme, precision_dr, precision_dme, recall_dr, recall_dme, f1score_dr, f1score_dme [0.9066666666666666, 0.8829788843633278, 0.7676767676767676, 0.7402146464646465, 0.8444444444444444, 0.709198018476369, 0.8042328042328042, 0.7238129082478763]
se, sp 0.8466666666666667 0.8444444444444444
time 3.96866512298584


For visual representation of Summary files

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir "/content/drive/MyDrive/Trained"

In [ ]:
# # !pip install Pillow==8.0.1
# from PIL import Image
# import glob
# import xlrd
# import numpy as np
# import scipy.misc
# import os
# import os.path
# import sys
# if sys.version_info[0] == 2:
#     import cPickle as pickle
# else:
#     import pickle
# import torch.utils.data as data
# import cv2

In [ ]:
# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225])
# tra_test = transforms.Compose([
#             transforms.Resize(256),
#             transforms.CenterCrop(224),
#             transforms.ToTensor(),
#             normalize])
# model = resnet50.resnet50(num_classes=  2, multitask=  True, liu=  False,
#                 chen=  False, CAN_TS=  False, crossCBAM=  True,
#                 crosspatialCBAM =   False,  choice=  False)
# model.load_state_dict(torch.load('/content/drive/MyDrive/Trained/CANet(1).pth'))
# print("model loaded")
# model.eval()
# %cd "/content/drive/MyDrive/Base/Base/"
# img = np.array(Image.open("20051020_45110_0100_PP.tif"))
# img_cv = cv2.imread('20051020_45110_0100_PP.tif')
# print("Image: ",img.size)
# print("cv2 Image ",img_cv.shape)
# im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# cv2.imwrite('/content/drive/MyDrive/20051020_45110_0100_PP.tif',im_rgb)
# # img = Image.open('/content/drive/MyDrive/20051020_45110_0100_PP.tif')
# # image = img.convert('RGB')
# # img.show()
# img = Image.fromarray(im_rgb)
# #img = img.convert("RGB")
# print("Image mode: ",img.mode)
# print("Image: ",img.size)
# red, green, blue = img.split()
# print("r mode: ",red.mode)
# print("g mode: ",green.mode)
# print("b mode: ",blue.mode)
# r, g, b = img.getpixel((1, 1))
# print(r, g, b)
# new_img = Image.merge("RGB",(red,green,blue))
# print("new size " ,new_img.size)
# imageTra = tra_test(new_img)
# img_loader = torch.utils.data.DataLoader(
#         imageTra,
#         batch_size=  1, shuffle=False,
#         num_workers=  4, pin_memory=True)
# dataiter = iter(img_loader)
# image= dataiter.next()
# print("Image shape:  ",image.shape)
# output = model(image)
# acc_dr = output[0]
# acc_dme = output[1]
# print("DR_stage : ", acc_dr)
# print("DME_stage : ", acc_dme)